<a href="https://colab.research.google.com/github/mrsamsonn/fbprophet-yfinance-next_hour_prediction/blob/main/fb_prophet_forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
#!pip install pystan~=2.14
!pip install prophet

In [136]:
%pip install yfinance

In [266]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

In [274]:
#date
past = (date.today()-timedelta(days=1))
print("date:", past)

date: 2023-09-13


In [275]:
data = yf.download('BTC-USD', start=past, interval='5m')
data.reset_index(inplace=True)
data['Datetime'] = pd.to_datetime(data['Datetime']) - timedelta(hours=7)
data['Datetime'] = data['Datetime'].dt.strftime('%Y/%m/%d %H:%M:%S')

[*********************100%%**********************]  1 of 1 completed


In [276]:
data.reset_index(inplace=True)
data = data[['Datetime','Close']]
data.columns = ['ds','y']
data

,ds,y
0,2023/09/12 17:00:00,25839.136719
1,2023/09/12 17:05:00,25823.312500
2,2023/09/12 17:10:00,25876.541016
3,2023/09/12 17:15:00,25889.068359
4,2023/09/12 17:20:00,25889.763672
...,...,...
544,2023/09/14 14:25:00,26612.398438
545,2023/09/14 14:30:00,26602.441406
546,2023/09/14 14:35:00,26605.367188
547,2023/09/14 14:40:00,26605.984375


In [277]:
from prophet import Prophet

In [278]:
prophet = Prophet(daily_seasonality=False)
prophet.add_seasonality(name='hourly', period=0.0416, fourier_order=1)
#prophet = Prophet()
prophet.fit(data)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjbyduzny/8vhg6xaf.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjbyduzny/85i990ul.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=7070', 'data', 'file=/tmp/tmpjbyduzny/8vhg6xaf.json', 'init=/tmp/tmpjbyduzny/85i990ul.json', 'output', 'file=/tmp/tmpjbyduzny/prophet_modeljf6l2afh/prophet_model-20230914214816.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
21:48:16 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:48:16 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [279]:
future_dates = prophet.make_future_dataframe(periods=1)
predictions = prophet.predict(future_dates)

In [282]:
from prophet.plot import plot_plotly

fig = plot_plotly(prophet, predictions)
fig.show()